In [ ]:
# Imports
import numpy as np                               
import pandas as pd                        
import matplotlib.pyplot as plt                  
import cv2             
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from PIL import Image
from skimage import exposure
import os                                        
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical          
from tensorflow.keras.models import load_model                                 
import warnings

import sys

In [ ]:
print(tf.__version__)

In [ ]:
output_path = '/gpfs/space/home/markusha/project/output/dataframes/'

In [ ]:
# Load dataframe from pickled pandas object
dataframe1 = pd.read_pickle(output_path + 'Kaggle_data_fixed.pkl')
dataframe2 = pd.read_pickle(output_path + 'Larssen_data1.pkl')
dataframe3 = pd.read_pickle(output_path + 'Larssen_data2.pkl')
dataframe4 = pd.read_pickle(output_path + 'GTSRB_data.pkl')
dataframe5 = pd.read_pickle(output_path + 'Belgium_data_fixed.pkl')

In [ ]:
training_data = dataframe1[['sign','img_path']].append(dataframe2[['sign','img_path']]).append(dataframe3[['sign','img_path']]).append(dataframe4[['sign','img_path']]).append(dataframe5[['sign','img_path']])

In [ ]:
training_data['sign'].value_counts()

In [ ]:
data = []
labels = np.array([])
data_clahe = []

In [ ]:
for _, image_file in training_data.iterrows():
    image = cv2.imread(image_file['img_path'])
    #resizing
    image = cv2.resize(image, (50,50), interpolation = cv2.INTER_AREA)
    image = np.array(image)
    # using clahe algorithm
    image_p = exposure.equalize_adapthist(image, clip_limit=0.1)
    data_clahe.append(image_p)
    # data = np.append(data,image)
    data.append(image)
    labels = np.append(labels, image_file['sign'])
data_clahe = np.array(data_clahe)

In [ ]:
labels

In [ ]:
labels = list(map(lambda x: x.replace(' ', ''), labels))

In [ ]:
print((np.unique(labels)))
classes = len(np.unique(labels))

In [ ]:
# We need to One-Hot Encode the labels
char_to_int = dict((c, i) for i, c in enumerate(np.unique(labels)))
labels_integer_encoded = [char_to_int[char] for char in labels]
print(char_to_int)
labels_1hot = to_categorical(labels_integer_encoded)


In [ ]:
# shuffeling data
from sklearn.utils import shuffle

data_clahe, labels_1hot = shuffle(np.array(data_clahe), np.array(labels_1hot))

# predicting with CLAHE data
X_train, X_test, y_train, y_test = train_test_split(data_clahe, labels_1hot, test_size=0.2, random_state=69)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
classes = y_train.shape[1]
print("# classes: " + str(classes))

In [ ]:
model2 = Sequential()
inputShape = X_train.shape[1:]
activation_method1 = "relu"
activation_method2 = "softmax"
padding_method = "same"
model2.add(Conv2D(8, (5, 5), padding=padding_method,input_shape=inputShape))
model2.add(Activation(activation_method1))
model2.add(BatchNormalization(axis=-1))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Conv2D(16, (3, 3), padding=padding_method))
model2.add(Activation(activation_method1))
model2.add(BatchNormalization(axis=-1))
model2.add(Conv2D(16, (3, 3), padding=padding_method))
model2.add(Activation(activation_method1))
model2.add(BatchNormalization(axis=-1))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Conv2D(32, (3, 3), padding=padding_method))
model2.add(Activation(activation_method))
model2.add(BatchNormalization(axis=-1))
model2.add(Conv2D(32, (3, 3), padding=padding_method))
model2.add(Activation(activation_method1))
model2.add(BatchNormalization(axis=-1))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Flatten())
model2.add(Dense(128))
model2.add(Activation(activation_method1))
model2.add(BatchNormalization())
model2.add(Dropout(0.5))
model2.add(Flatten())
model2.add(Dense(128))
model2.add(Activation(activation_method1))
model2.add(BatchNormalization())
model2.add(Dropout(0.5))
model2.add(Dense(classes))
model2.add(Activation(activation_method2))




In [ ]:
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model2.summary()

In [ ]:
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model2.fit(X_train, y_train, batch_size=64, epochs=40, validation_data=(X_test, y_test))

In [ ]:
model2.save("Trafic_signs_model_final2.h5")
#plotting graphs for accuracy 
plt.figure(0)
plt.plot(history.history['accuracy'], label='training accuracy')
plt.plot(history.history['val_accuracy'], label='val accuracy')
plt.title('Accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()
#plotting graphs for loss 
plt.figure(1)
plt.plot(history.history['loss'], label='training loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.title('Loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

In [ ]:
# model2 = load_model('Trafic_signs_model_final.h5')

In [ ]:
predictions = model2.predict(data_clahe)

In [ ]:
predictions.shape

In [ ]:
labels_1hot.shape

In [ ]:
int_to_char = {k:v for k,v in zip(char_to_int.values(),char_to_int.keys())}

In [ ]:
predictions_integer = [np.argmax(x) for x in predictions]

In [ ]:
labels_predicted = [int_to_char[integer] for integer in list(predictions_integer)]

In [ ]:
labels_integer_encoded = [np.argmax(x) for x in labels_1hot]

In [ ]:
labels_true = [int_to_char[integer] for integer in labels_integer_encoded]

In [ ]:
# def prediction_to_label(model, X_test, char_to_int):
#     predictions = model.predict(X_test)
#     predictions = [np.argmax(x) for x in predictions]
#     int_to_char = {k:v for k,v in zip(char_to_int.values(),char_to_int.keys())}
#     labels_predicted = [int_to_char[integer] for integer in predictions]
#     return labels_predicted

In [ ]:
labels_predicted

In [ ]:
# import pickle

In [ ]:
# a_file = open("char_to_int.pkl", "wb")
# pickle.dump(char_to_int, a_file)
# a_file.close()

## open as 
#a_file = open("char_to_int.pkl", "rb")
#output = pickle.load(a_file)
#print(output)


In [ ]:
# Description

# # Let's predict the whole dataset
# data = np.array(data_clahe)

In [ ]:
# predictions = model2.predict(data)

In [ ]:
# predictions2 = [np.argmax(x) for x in predictions] #index

In [ ]:
 prediction_strength = [predictions[i][j] for i,j in enumerate(predictions_integer)]

In [ ]:
df = pd.DataFrame({'label' : labels_true, 'predicted':labels_predicted, 'strength':prediction_strength})

In [ ]:
df['correct'] = list(df['label'] ==df['predicted'])

In [ ]:
df

In [ ]:
df[df.predicted=='PRIORITY_ROAD'][df.correct == False].size

In [ ]:
df[df.predicted=='PRIORITY_ROAD'].size

In [ ]:
df[df['correct']][['label', 'strength']].groupby('label',as_index=False).mean().sort_values('strength',ascending = False)

In [ ]:
df[df['correct']][['label', 'strength']].groupby('label',as_index=False).mean().sort_values('strength',ascending = True).head(3)

In [ ]:
df[df['correct'] == False][['label', 'strength']].groupby('label',as_index=False).mean().sort_values('strength',ascending = True).head(3)

In [ ]:
df[df['correct'] == False][['label', 'strength']].groupby('label',as_index=False).mean().sort_values('strength',ascending = False)

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
predictions.shape

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
y_pred=np.argmax(predictions, axis=1)

In [ ]:
list(char_to_int.keys())

In [ ]:
y_tester = np.argmax(labels_1hot, axis=1)

In [ ]:
print(classification_report(y_tester, y_pred, target_names =list(char_to_int.keys())))

In [ ]:
index = 101
print("Label: " + labels_true[index])
print("Predicted: " + labels_predicted[index])
plt.imshow(data_clahe[index])